In [42]:
# !pip install kaleido==0.1.*
# !pip install wandb
# !pip install datashader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# plotly
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import kaleido
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error,mean_squared_error

# use ggplot2 template
pio.templates.default = 'ggplot2'

In [2]:
solar_df = pd.read_csv('processed_all_sites_HS1e0_df_not_imputed_R_channel.csv', parse_dates=['Datetime'], index_col = 'Datetime')

In [43]:
## no cloud

start_train_date = '2022-02-05'
end_train_date = '2022-10-31'
start_test_date = '2022-11-01' #'2022-02-05'
end_test_date = '2023-02-28'
p,d,q,P,D,Q,T = 1,0,1,0,1,1,24
step_of_forecast = 8
total_lag = D*T+p

no_site = [e for e in range(1,5)]
no_site = no_site + [e for e in range(6,13)]
no_site = no_site + [e for e in range(14,31)]
no_site = no_site + [e for e in range(34,38)]
no_site = no_site + [e for e in range(39,44)]
no_site = no_site + [e for e in range(46,54)]
no_site = no_site + [e for e in range(55,57)]

y_merge = pd.DataFrame()
I1 = pd.DataFrame()
I2 = pd.DataFrame()
I3 = pd.DataFrame()
I4 = pd.DataFrame()
I5 = pd.DataFrame()
I6 = pd.DataFrame()
I7 = pd.DataFrame()
I8 = pd.DataFrame()

for i in no_site:
    
    print(i)
    
    ## set up data
    site = solar_df[solar_df['site'] == i]
    site_train_test = site[['iclr','I']]
    site_train_test.dropna(inplace = True)
    X = site_train_test[['iclr']]
    y = site_train_test[['I']].loc[start_train_date:end_test_date]


    X_train, y_train = X.loc[start_train_date:end_train_date], np.asarray(y.loc[start_train_date:end_train_date])
    X_test, y_test = X.loc[start_test_date:end_test_date], np.asarray(y.loc[start_test_date:end_test_date])
#     y_endog = np.asarray(y.loc[start_train_date:end_endog_date])
    y_endog = y_train[-total_lag+1:]

    I_1_step = []
    I_2_step = []
    I_3_step = []
    I_4_step = []
    I_5_step = []
    I_6_step = []
    I_7_step = []
    I_8_step = []

    endo = y_endog[-total_lag+1:]
    ## dummy variable since the 'extend' function needs to put the argument 'exog'
    exo = np.array(X_train.iloc[-total_lag+1:])


    ## model fitting
    model = sm.tsa.statespace.SARIMAX(y_train, exog=np.array(X_train), order = (p,d,q), seasonal_order = (P,D,Q,T))
    result = model.fit()

    ## forecasting
    for j in range(1,len(y.loc[start_test_date:end_test_date])-step_of_forecast):
      y_pred_1 = result.forecast(steps = 8, exog=X_test.iloc[-len(y.loc[start_test_date:end_test_date])+j:
                                                             -len(y.loc[start_test_date:end_test_date])+j+step_of_forecast])
      I_1_step.append(y_pred_1[0])
      I_2_step.append(y_pred_1[1])
      I_3_step.append(y_pred_1[2])
      I_4_step.append(y_pred_1[3])
      I_5_step.append(y_pred_1[4])
      I_6_step.append(y_pred_1[5])
      I_7_step.append(y_pred_1[6])
      I_8_step.append(y_pred_1[7])

      endo = np.concatenate((endo[-total_lag+1:],y_test[j:j+1]))
      exo = np.concatenate((exo[-total_lag+1:],np.array(X_test.iloc[j:j+1])))
      result = result.extend(endog = endo, exog = exo)
        
    I_1_step = np.array(I_1_step)
    I_2_step = np.array(I_2_step)
    I_3_step = np.array(I_3_step)
    I_4_step = np.array(I_4_step)
    I_5_step = np.array(I_5_step)
    I_6_step = np.array(I_6_step)
    I_7_step = np.array(I_7_step)
    I_8_step = np.array(I_8_step)
    
    I_1_step[I_1_step < 0] = 0
    I_2_step[I_2_step < 0] = 0
    I_3_step[I_3_step < 0] = 0
    I_4_step[I_4_step < 0] = 0
    I_5_step[I_5_step < 0] = 0
    I_6_step[I_6_step < 0] = 0
    I_7_step[I_7_step < 0] = 0
    I_8_step[I_8_step < 0] = 0
    
    y_forecast = pd.DataFrame()
    y_forecast['forecast_1_step'] = I_1_step
    y_forecast['forecast_2_step'] = I_2_step
    y_forecast['forecast_3_step'] = I_3_step
    y_forecast['forecast_4_step'] = I_4_step
    y_forecast['forecast_5_step'] = I_5_step
    y_forecast['forecast_6_step'] = I_6_step
    y_forecast['forecast_7_step'] = I_7_step
    y_forecast['forecast_8_step'] = I_8_step
    y_forecast['Datetime'] = np.array(y.iloc[-len(y.loc[start_test_date:end_test_date]):-step_of_forecast-1].index)
    y_forecast = y_forecast.set_index(y_forecast['Datetime'])
    y_forecast.drop('Datetime', axis=1, inplace=True)



    y_test_all = site[['I_lead_1step','I_lead_2step','I_lead_3step','I_lead_4step','I_lead_5step','I_lead_6step','I_lead_7step','I_lead_8step']]
    y_test_all = y_test_all.loc[start_test_date:end_test_date]
    error_1_step = pd.merge(y_test_all['I_lead_1step'].dropna(), y_forecast['forecast_1_step'], how = 'inner', left_index = True, right_index = True)
    error_2_step = pd.merge(y_test_all['I_lead_2step'].dropna(), y_forecast['forecast_2_step'], how = 'inner', left_index = True, right_index = True)
    error_3_step = pd.merge(y_test_all['I_lead_3step'].dropna(), y_forecast['forecast_3_step'], how = 'inner', left_index = True, right_index = True)
    error_4_step = pd.merge(y_test_all['I_lead_4step'].dropna(), y_forecast['forecast_4_step'], how = 'inner', left_index = True, right_index = True)
    error_5_step = pd.merge(y_test_all['I_lead_5step'].dropna(), y_forecast['forecast_5_step'], how = 'inner', left_index = True, right_index = True)
    error_6_step = pd.merge(y_test_all['I_lead_6step'].dropna(), y_forecast['forecast_6_step'], how = 'inner', left_index = True, right_index = True)
    error_7_step = pd.merge(y_test_all['I_lead_7step'].dropna(), y_forecast['forecast_7_step'], how = 'inner', left_index = True, right_index = True)
    error_8_step = pd.merge(y_test_all['I_lead_8step'].dropna(), y_forecast['forecast_8_step'], how = 'inner', left_index = True, right_index = True)
    
    error_1_step = error_1_step.between_time('06:29','16:59')
    error_2_step = error_2_step.between_time('06:29','16:29')
    error_3_step = error_3_step.between_time('06:29','15:59')
    error_4_step = error_4_step.between_time('06:29','15:29')
    error_5_step = error_5_step.between_time('06:29','14:59')
    error_6_step = error_6_step.between_time('06:29','14:29')
    error_7_step = error_7_step.between_time('06:29','13:59')
    error_8_step = error_8_step.between_time('06:29','13:29')

    y_pre_merge = pd.DataFrame()
    y_pre_merge = pd.merge(y_pre_merge,error_1_step,how = 'outer', left_index = True, right_index = True)
    y_pre_merge = pd.merge(y_pre_merge,error_2_step,how = 'outer', left_index = True, right_index = True)
    y_pre_merge = pd.merge(y_pre_merge,error_3_step,how = 'outer', left_index = True, right_index = True)
    y_pre_merge = pd.merge(y_pre_merge,error_4_step,how = 'outer', left_index = True, right_index = True)
    y_pre_merge = pd.merge(y_pre_merge,error_5_step,how = 'outer', left_index = True, right_index = True)
    y_pre_merge = pd.merge(y_pre_merge,error_6_step,how = 'outer', left_index = True, right_index = True)
    y_pre_merge = pd.merge(y_pre_merge,error_7_step,how = 'outer', left_index = True, right_index = True)
    y_pre_merge = pd.merge(y_pre_merge,error_8_step,how = 'outer', left_index = True, right_index = True)
    y_pre_merge['site'] = i
#     y_pre_merge.drop(['I_lead_1step','I_lead_2step','I_lead_3step','I_lead_4step','I_lead_5step','I_lead_6step','I_lead_7step','I_lead_8step'],axis = 1, inplace = True)
    y_merge = pd.concat([y_merge, y_pre_merge])


    error_1_step['site'] = i
    error_2_step['site'] = i
    error_3_step['site'] = i
    error_4_step['site'] = i
    error_5_step['site'] = i
    error_6_step['site'] = i
    error_7_step['site'] = i
    error_8_step['site'] = i
    I1 = pd.concat([I1,error_1_step])
    I2 = pd.concat([I2,error_2_step])
    I3 = pd.concat([I3,error_3_step])
    I4 = pd.concat([I4,error_4_step])
    I5 = pd.concat([I5,error_5_step])
    I6 = pd.concat([I6,error_6_step])
    I7 = pd.concat([I7,error_7_step])
    I8 = pd.concat([I8,error_8_step])

#   return y_merge
#   return I1,I2,I3,I4,I5,I6,I7,I8

1


C:\Users\User\AppData\Local\Temp\ipykernel_6544\3303734108.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



2


C:\Users\User\AppData\Local\Temp\ipykernel_6544\3303734108.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



3


...

C:\Users\User\AppData\Local\Temp\ipykernel_6544\3303734108.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



55


C:\Users\User\AppData\Local\Temp\ipykernel_6544\3303734108.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



56


C:\Users\User\AppData\Local\Temp\ipykernel_6544\3303734108.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [44]:
y_merge.to_csv('y_merge.csv')

In [47]:
def get_metric(y_merge):
    RMSE_list = []
    MAE_list = []
    NRMSE_list = []
    NMAE_list = []
    
    for i in range(1,9):
        pre_df = y_merge[[f'I_lead_{i}step',f'forecast_{i}_step']].dropna()
        MAE = mean_absolute_error(pre_df[f'I_lead_{i}step'],pre_df[f'forecast_{i}_step'])
        RMSE = mean_squared_error(pre_df[f'I_lead_{i}step'],pre_df[f'forecast_{i}_step'])**0.5
        MAE_list.append(MAE)
        RMSE_list.append(RMSE)
        NMAE_list.append(MAE/pre_df[f'I_lead_{i}step'].mean()*100)
        NRMSE_list.append(RMSE/pre_df[f'I_lead_{i}step'].mean()*100)
    metric_list = [MAE_list,RMSE_list,NMAE_list,NRMSE_list]
    metric_df = pd.DataFrame(metric_list,index = ['MAE','RMSE','NMAE','NRMSE'])
    metric_df = metric_df.rename(columns = {0:'t+30',1:'t+60',2:'t+90',3:'t+120'
                                           ,4:'t+150',5:'t+30',6:'t+30',7:'t+30'})
    
    return MAE_list,RMSE_list,NMAE_list,NRMSE_list,metric_df

In [48]:
MAE_list,RMSE_list,NMAE_list,NRMSE_list,metric_df = get_metric(y_merge)
MAE_cloud_list,RMSE_cloud_list,NMAE_cloud_list,NRMSE_cloud_list,metric_cloud_df = get_metric(y_merge_cloud)

In [49]:
metric_df

,t+30,t+60,t+90,t+120,t+150,t+30,t+30,t+30
MAE,68.328743,90.662231,106.814749,118.659194,127.330181,133.161393,136.576016,137.693743
RMSE,97.342371,118.962131,134.668495,146.663457,155.984188,162.560380,166.620124,168.364670
NMAE,14.851088,18.871241,21.412516,23.086523,24.255852,25.070533,25.655628,26.067564
NRMSE,21.157131,24.761833,26.996190,28.535077,29.714317,30.605533,31.299375,31.874046


In [50]:
metric_cloud_df

,t+30,t+60,t+90,t+120,t+150,t+30,t+30,t+30
MAE,165.779914,161.445161,166.097983,172.316589,178.137823,182.701329,185.426472,187.080149
RMSE,206.812627,201.686820,207.252017,214.796508,222.061321,227.861924,231.481525,233.827701
NMAE,35.670646,33.295363,33.007790,33.274062,33.660933,34.112311,34.658874,35.342313
NRMSE,44.499601,41.594532,41.186117,41.476868,41.960719,42.544282,43.267226,44.173644


In [51]:
def get_mae_dict_hour(y_merge,step):
  """ Get mean of Absolute Difference for each our in list
  Agrs :
  DataFrame with Hour of Day column
  Return :
  Dictionary
  """
  mae_dict = {}
    
  df = pd.DataFrame()
  df['error'] = abs(y_merge[f'I_lead_{step}step']-y_merge[f'forecast_{step}_step'])
  df.index = df.index + pd.Timedelta(minutes=step*30)
  
  for hour in pd.date_range('7:00:00', '17:00:00', freq='30min').time[step-1:]:
    mae = df['error'][df.index.time == hour].dropna().mean()
    mae_dict[hour] = mae
  return mae_dict

mae_dict_hour_cloud_8 = get_mae_dict_hour(y_merge_cloud,8)
mae_dict_hour_8 = get_mae_dict_hour(y_merge,8)

In [52]:
def MAE(mae_dict_hour_step,step):  
  """Get MAE for each hour of the day
  Args : 
  MAE dict from get_mae_dict_hour and that k-step
  Return :
  Graph and file
  """
  from plotly.subplots import make_subplots
  import plotly.graph_objects as go
  import plotly.io as pio
  pio.templates.default = 'plotly_white'
  day_time = pd.date_range('7:00:00', '17:00:00', freq='30min').time[step-1:]
  mae_error_list = [mae_dict_hour_step[hour] for hour in day_time]
  mae_error_array = np.array(mae_error_list)
  
  fig = go.Figure()
  fig.add_trace(go.Bar(x=day_time, y=mae_error_array, name = 'MAE'))
  fig.update_layout(xaxis_title='<b> Hour of Day </b>',
                      yaxis_title='<b> MAE [W/m2] </b>')
  fig.update_yaxes(title_text='MAE [W/m<sup>2</sup>]',range=[0, 250], 
                    showline=True, linewidth=1, linecolor='black',mirror=True,showgrid=True)
  fig.update_xaxes(title_text='<b> Time of forecasted value </b>',tickangle= 45,
                     showline=True, linewidth=1, linecolor='black',mirror=True)
  fig.show()
  pio.write_image(fig, 'Hourly_MAE_SARIMAX.pdf')
  return None

MAE(mae_dict_hour_8,8)

In [30]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from datetime import timedelta


def plot_and_result(y_merge, step, site, legend_status):
    
  y_merge = y_merge[y_merge['site'] == 1]
  y_merge = y_merge[[f'I_lead_{step}step',f'forecast_{step}_step']].dropna().iloc[:(22-step)*3]
  y_true = y_merge[f'I_lead_{step}step'].ravel()
  y_pred = y_merge[f'forecast_{step}_step'].ravel()
  dates = y_merge.index + timedelta(minutes = 30 * step)

  dates_list = [dt.strftime('%m-%d %H:%M') for dt in dates]
  
  r_2 = r2_score(y_true, y_pred)
  rmse = mean_squared_error(y_true, y_pred) ** 0.5
  mae = mean_absolute_error(y_true,y_pred)

  fig = go.Figure()
  fig.add_trace(go.Scatter(x=dates_list,
                            y=y_true,
                            name="Actual",
                            mode='lines+markers',
                           marker=dict(symbol='x', size=8),
                            line=dict(color='royalblue', width=2)
                           ,showlegend=legend_status
                            ))

  fig.add_trace(go.Scatter(x=dates_list,
                            y=y_pred,
                            name="Forecasted",
                           mode='lines+markers',
                           marker=dict(symbol='circle-open', size=8),
                            line=dict(color='firebrick', width=2)
                           ,showlegend=legend_status   
                            ))

  fig.update_layout(
                    title = f'Site {site}',
                    xaxis_title='Date',
                    yaxis_title='Irradiance[W/m2]',
                    xaxis=dict(title_font=dict(size=20),showgrid=False,tickmode='linear', dtick=21, tickfont=dict(size=16)),
                    yaxis=dict(title_font=dict(size=20),showgrid=False),
                    legend=dict(x=0.75,y=1,traceorder='normal', font=dict(size=12)),
                    font=dict(size=18))

  fig.update_yaxes(title_text='Irradiance [W/m<sup>2</sup>]',range=[-50, 1100], 
                    showline=True, linewidth=1, linecolor='black',mirror=True,showgrid=True)
  fig.update_xaxes(title_text='<b> Date </b>',tickangle= 360,
                     showline=True, linewidth=1, linecolor='black',mirror=True)
  fig.show()
  result = [rmse, mae, r_2]
  pio.write_image(fig, f'{step}_step_forecast_SARIMAX_site{site}.jpg')
  pio.write_image(fig, f'{step}_step_forecast_SARIMAX_site{site}.pdf')
  pio.write_image(fig, f'{step}_step_forecast_SARIMAX_site{site}.svg')
#   pio.write_image(fig, f'{step}_step_forecast_SARIMAX_site{site}.eps')
  return result

plot_and_result(y_merge, step = 1, site = 48, legend_status = True)

[66.78049904155299, 58.393224877808805, 0.933827995509367]

In [45]:
# include cloud

start_train_date = '2022-02-05'
end_train_date = '2022-10-31'
start_test_date = '2022-11-01' #'2022-02-05'
end_test_date = '2023-02-28'
p,d,q,P,D,Q,T = 1,0,1,0,1,1,24
step_of_forecast = 8
total_lag = D*T+p

no_site = [e for e in range(1,5)]
no_site = no_site + [e for e in range(6,13)]
no_site = no_site + [e for e in range(14,31)]
no_site = no_site + [e for e in range(34,38)]
no_site = no_site + [e for e in range(39,44)]
no_site = no_site + [e for e in range(46,54)]
no_site = no_site + [e for e in range(55,57)]

y_merge_cloud = pd.DataFrame()
I1_cloud = pd.DataFrame()
I2_cloud = pd.DataFrame()
I3_cloud = pd.DataFrame()
I4_cloud = pd.DataFrame()
I5_cloud = pd.DataFrame()
I6_cloud = pd.DataFrame()
I7_cloud = pd.DataFrame()
I8_cloud = pd.DataFrame()
for i in no_site:
    ## set up data
    print(i)
    site = solar_df[solar_df['site'] == i]
    site_train_test = site[['iclr','I','ci_est(t+1)','ci_est(t+2)','ci_est(t+3)',
                            'ci_est(t+4)','ci_est(t+5)','ci_est(t+6)','ci_est(t+7)','ci_est(t+8)','ci_center']]
    site_train_test.dropna(inplace = True)
    X = site_train_test[['iclr','ci_est(t+1)','ci_est(t+2)','ci_est(t+3)','ci_est(t+4)','ci_est(t+5)','ci_est(t+6)','ci_est(t+7)','ci_est(t+8)']]
    y = site_train_test[['I']].loc[start_train_date:end_test_date]
    X_for_fit = site_train_test[['iclr','ci_center']].loc[start_train_date:end_train_date]

    X_train, y_train = X.loc[start_train_date:end_train_date], np.asarray(y.loc[start_train_date:end_train_date])
    X_test, y_test = X.loc[start_test_date:end_test_date], np.asarray(y.loc[start_test_date:end_test_date])
#     y_endog = np.asarray(y.loc[start_train_date:end_endog_date])
    y_endog = y_train[-total_lag+1:]

    I_1_step = []
    I_2_step = []
    I_3_step = []
    I_4_step = []
    I_5_step = []
    I_6_step = []
    I_7_step = []
    I_8_step = []

    endo = y_endog[-total_lag+1:]
    exo = np.array(X_for_fit.iloc[-total_lag+1:])



    ## model fitting
    model = sm.tsa.statespace.SARIMAX(y_train, exog=np.array(X_for_fit), order = (p,d,q), seasonal_order = (P,D,Q,T))
    result = model.fit()

    ## forecasting
    for j in range(1,len(y.loc[start_test_date:end_test_date])-8):
      exog_for_result = np.array(X_test[['iclr']].iloc[-len(y.loc[start_test_date:end_test_date])+j:
                                                       -len(y.loc[start_test_date:end_test_date])+j+step_of_forecast])
      pre_CI = np.array(X_test[['ci_est(t+1)','ci_est(t+2)','ci_est(t+3)','ci_est(t+4)'
                                ,'ci_est(t+5)','ci_est(t+6)','ci_est(t+7)','ci_est(t+8)']].iloc[-len(y.loc[start_test_date:end_test_date])+j:
                                                                                                -len(y.loc[start_test_date:end_test_date])+j+1].T)
      exog_for_result = np.concatenate([exog_for_result,pre_CI],axis = 1)

      y_pred_1 = result.forecast(steps = 8, exog=exog_for_result)
      I_1_step.append(y_pred_1[0])
      I_2_step.append(y_pred_1[1])
      I_3_step.append(y_pred_1[2])
      I_4_step.append(y_pred_1[3])
      I_5_step.append(y_pred_1[4])
      I_6_step.append(y_pred_1[5])
      I_7_step.append(y_pred_1[6])
      I_8_step.append(y_pred_1[7])

      endo = np.concatenate((endo[-total_lag+1:],y_test[j:j+1]))
      exo = np.concatenate((exo[-total_lag+1:],np.array(X_for_fit.iloc[j:j+1])))
      result = result.extend(endog = endo, exog = exo)
        
    I_1_step = np.array(I_1_step)
    I_2_step = np.array(I_2_step)
    I_3_step = np.array(I_3_step)
    I_4_step = np.array(I_4_step)
    I_5_step = np.array(I_5_step)
    I_6_step = np.array(I_6_step)
    I_7_step = np.array(I_7_step)
    I_8_step = np.array(I_8_step)
    
    I_1_step[I_1_step < 0] = 0
    I_2_step[I_2_step < 0] = 0
    I_3_step[I_3_step < 0] = 0
    I_4_step[I_4_step < 0] = 0
    I_5_step[I_5_step < 0] = 0
    I_6_step[I_6_step < 0] = 0
    I_7_step[I_7_step < 0] = 0
    I_8_step[I_8_step < 0] = 0
    
    y_forecast = pd.DataFrame()
    y_forecast['forecast_1_step'] = I_1_step
    y_forecast['forecast_2_step'] = I_2_step
    y_forecast['forecast_3_step'] = I_3_step
    y_forecast['forecast_4_step'] = I_4_step
    y_forecast['forecast_5_step'] = I_5_step
    y_forecast['forecast_6_step'] = I_6_step
    y_forecast['forecast_7_step'] = I_7_step
    y_forecast['forecast_8_step'] = I_8_step
    y_forecast['Datetime'] = np.array(y.iloc[-len(y.loc[start_test_date:end_test_date]):-step_of_forecast-1].index)
    y_forecast = y_forecast.set_index(y_forecast['Datetime'])
    y_forecast.drop('Datetime', axis=1, inplace=True)



    y_test_all = site[['I_lead_1step','I_lead_2step','I_lead_3step','I_lead_4step','I_lead_5step','I_lead_6step','I_lead_7step','I_lead_8step']]
    y_test_all = y_test_all.loc[start_test_date:end_test_date]
    error_1_step = pd.merge(y_test_all['I_lead_1step'].dropna(), y_forecast['forecast_1_step'], how = 'inner', left_index = True, right_index = True)
    error_2_step = pd.merge(y_test_all['I_lead_2step'].dropna(), y_forecast['forecast_2_step'], how = 'inner', left_index = True, right_index = True)
    error_3_step = pd.merge(y_test_all['I_lead_3step'].dropna(), y_forecast['forecast_3_step'], how = 'inner', left_index = True, right_index = True)
    error_4_step = pd.merge(y_test_all['I_lead_4step'].dropna(), y_forecast['forecast_4_step'], how = 'inner', left_index = True, right_index = True)
    error_5_step = pd.merge(y_test_all['I_lead_5step'].dropna(), y_forecast['forecast_5_step'], how = 'inner', left_index = True, right_index = True)
    error_6_step = pd.merge(y_test_all['I_lead_6step'].dropna(), y_forecast['forecast_6_step'], how = 'inner', left_index = True, right_index = True)
    error_7_step = pd.merge(y_test_all['I_lead_7step'].dropna(), y_forecast['forecast_7_step'], how = 'inner', left_index = True, right_index = True)
    error_8_step = pd.merge(y_test_all['I_lead_8step'].dropna(), y_forecast['forecast_8_step'], how = 'inner', left_index = True, right_index = True)
    
    error_1_step = error_1_step.between_time('06:29','16:59')
    error_2_step = error_2_step.between_time('06:29','16:29')
    error_3_step = error_3_step.between_time('06:29','15:59')
    error_4_step = error_4_step.between_time('06:29','15:29')
    error_5_step = error_5_step.between_time('06:29','14:59')
    error_6_step = error_6_step.between_time('06:29','14:29')
    error_7_step = error_7_step.between_time('06:29','13:59')
    error_8_step = error_8_step.between_time('06:29','13:29')

    y_pre_merge_cloud = pd.DataFrame()
    y_pre_merge_cloud = pd.merge(y_pre_merge_cloud,error_1_step,how = 'outer', left_index = True, right_index = True)
    y_pre_merge_cloud = pd.merge(y_pre_merge_cloud,error_2_step,how = 'outer', left_index = True, right_index = True)
    y_pre_merge_cloud = pd.merge(y_pre_merge_cloud,error_3_step,how = 'outer', left_index = True, right_index = True)
    y_pre_merge_cloud = pd.merge(y_pre_merge_cloud,error_4_step,how = 'outer', left_index = True, right_index = True)
    y_pre_merge_cloud = pd.merge(y_pre_merge_cloud,error_5_step,how = 'outer', left_index = True, right_index = True)
    y_pre_merge_cloud = pd.merge(y_pre_merge_cloud,error_6_step,how = 'outer', left_index = True, right_index = True)
    y_pre_merge_cloud = pd.merge(y_pre_merge_cloud,error_7_step,how = 'outer', left_index = True, right_index = True)
    y_pre_merge_cloud = pd.merge(y_pre_merge_cloud,error_8_step,how = 'outer', left_index = True, right_index = True)
    y_pre_merge_cloud['site'] = i
#     y_pre_merge_cloud.drop(['I_lead_1step','I_lead_2step','I_lead_3step','I_lead_4step','I_lead_5step','I_lead_6step','I_lead_7step','I_lead_8step'],axis = 1, inplace = True)
    y_merge_cloud = pd.concat([y_merge_cloud, y_pre_merge_cloud])


    error_1_step['site'] = i
    error_2_step['site'] = i
    error_3_step['site'] = i
    error_4_step['site'] = i
    error_5_step['site'] = i
    error_6_step['site'] = i
    error_7_step['site'] = i
    error_8_step['site'] = i
    I1_cloud = pd.concat([I1_cloud,error_1_step])
    I2_cloud = pd.concat([I2_cloud,error_2_step])
    I3_cloud = pd.concat([I3_cloud,error_3_step])
    I4_cloud = pd.concat([I4_cloud,error_4_step])
    I5_cloud = pd.concat([I5_cloud,error_5_step])
    I6_cloud = pd.concat([I6_cloud,error_6_step])
    I7_cloud = pd.concat([I7_cloud,error_7_step])
    I8_cloud = pd.concat([I8_cloud,error_8_step])


1


C:\Users\User\AppData\Local\Temp\ipykernel_6544\2736913982.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



2


C:\Users\User\AppData\Local\Temp\ipykernel_6544\2736913982.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

D:\Anacondas\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



3


...

C:\Users\User\AppData\Local\Temp\ipykernel_6544\2736913982.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



53


C:\Users\User\AppData\Local\Temp\ipykernel_6544\2736913982.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\User\AppData\Local\Temp\ipykernel_6544\2736913982.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



55
56


C:\Users\User\AppData\Local\Temp\ipykernel_6544\2736913982.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [46]:
y_merge_cloud.to_csv('y_merge_cloud.csv')